In [1]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [7]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
stats_file = 'stats_manyall' # 'stats_samples'
target_file = 'target_manyall' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [8]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)
    
with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [9]:
def downselect(stats_inp, pair_knots, edens_knots):
    
    # find idices of knots
    pair_index = find_index(pair_knots, stats_inp['hyperparams']['pair'])
    edens_index = find_index(edens_knots, stats_inp['hyperparams']['edens'])
    
    # create boolean arrays with select indices set to True
    p_ix = np.array([True if i in pair_index else False for i in range(len(stats_inp['hyperparams']['pair']))])
    m_ix = np.array([True if i in edens_index else False for i in range(len(stats_inp['hyperparams']['edens']))])

    stats_out = select_nodes(stats_inp, p_ix, m_ix)
    
    return stats_out

In [44]:
pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
        3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
        4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
edens_knots = [ 2.5, 2.8663175, 3.1,  3.5,  4.9, 5.4604375]

In [45]:
stats = downselect(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])
stats_select_md = downselect(stats_md, pair_knots, edens_knots)
print(stats_select_md['hyperparams'])

{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 2.8663175000000001, 3.1000000000000001, 3.5, 4.9000000000000004, 5.4604375000000003]}
{u'pair': [2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003], u'edens': [2.5, 2.8663175000000001, 3.1000000000000001, 3.5, 4.9000000000000004, 5.4604375000000003]}


In [46]:
targets['md'] = targets_md['md']
stats['md'] = stats_select_md['md']

In [47]:
for key in sorted(targets.keys()):
    for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
        n = len(xyz)
        vol = np.linalg.det(box)
        corr = float(n*n)/vol
        ene.append([corr])

In [48]:
stats['bcc_npt_langevin_300K']['energy'][0][:]

[array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.32452672e-04,   2.40108437e-01,   1.49699018e+01,
          3.67830273e+01,   6.58053659e+01,   2.44033647e+02,
          4.40325883e+02,   6.80111008e+02,   1.13544979e+03,
          1.32771865e+03,   1.53444986e+03,   1.82571708e+03,
          1.89696652e+03,   1.96972937e+03]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          4.37184093e-15,   3.18476767e-07,   2.82775234e-02,
          9.17736776e-01,   9.14748666e+00,   1.69749763e+03,
          1.79543454e+04,   1.02110145e+05,   7.92918644e+05,
          1.48233268e+06,   2.64426772e+06,   5.29910283e+06,
          6.17592659e+06,   7.17934901e+06]),
 array([  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          9.35076567e-08,   2.01440000e-03,   1.80322649e+00,
          1.06606591e+01,   3.39604107e+01,   4.65546847e+02,
          1.51515052e+03,   3.61420678e+03,   1.00729590e+04,
          1.37729635e+04,   1.83956954e+

In [49]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.0
targets['eos_bcc']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [50]:
targets.keys()

['bcc_npt_langevin_3700K',
 'md',
 'vac_npt_langevin_2000K',
 'relax',
 'screw_111_npt_langevin_2000K',
 'bcc_npt_langevin_300K',
 'i111_npt_langevin_2000K',
 'eos',
 'fcc_npt_langevin_300K',
 'eos_fcc',
 'i110_npt_langevin_2000K',
 'liq_3700K',
 'liq_5000K',
 'eos_bcc',
 'bain']

In [51]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('md', 1.0),
 ('vac_npt_langevin_2000K', 1.0),
 ('relax', 1.0),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('eos', 0.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('liq_5000K', 1.0),
 ('eos_bcc', 0.0),
 ('bain', 0.0)]

In [52]:
start_params = []
# start_params.append([ -4.18353755e+00,  -9.63668936e-03,  0.0, -1.00171035e+02,
#          1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
#          2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
#          1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
#          1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
#         -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
#          4.64461212e+00,   3.23329113e-01,   5.82061842e-02, -0.1])
# start_params.append([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
#         -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
#         -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
#          2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
#         -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
#          2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
#         -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
#          5.51658454e-02,   1.58601447e+00])
# start_params.append([ -4.16739003e+00,  -4.97036785e-03,  -4.62426912e+02,
#          2.63469650e+02,   1.00253629e+01,   2.05151925e+01,
#         -1.36702712e+01,   2.29129977e+01,  -1.27210473e+01,
#          2.95533783e+00,   8.18470873e-01,  -7.89494754e-01,
#         -8.74455389e-01,   2.10177842e+00,  -9.22682795e-01,
#         -6.93852493e-02,  -1.52166575e+00,   1.40766925e+00,
#          1.52571934e+02,   3.37536873e-01,   6.89310740e-01,
#          4.87102227e-02,   2.79573955e+00])
# start_params.append([ -4.33954858e+00,  -1.02215197e-02,  0.0, -1.12590032e+02,
#          3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
#          2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
#          1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
#          1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
#         -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
#          4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
#          3.46319317e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02,   5.60287637e+00])

# start_params.append([-4.48347382e+00,  -7.78079006e-03,   2.92055760e+02,
#          1.84809430e+02,  -6.18705307e+01,   3.27735461e+01,
#         -1.36588262e+01,   2.11547368e+01,  -5.45394162e+00,
#          8.02998352e-01,   1.21949599e+00,  -9.43034331e-01,
#         -7.71285316e-01,   2.00564540e+00,  -8.93039729e-01,
#         -9.92574423e-03,  -1.55106037e+00,   1.39898785e+00,
#          2.54912062e+02,   3.19484582e+00,  -3.49947883e-01,
#          6.32970386e-02,   4.13612869e+00])

# start_params.append([ -4.41712815e+00,  -7.57807965e-03, 0.0, 2.79276814e+02,
#          1.74479561e+02,  -6.59620942e+01,   3.53977481e+01,
#         -1.40546374e+01,   2.11913467e+01,  -4.95603366e+00,
#          8.26392820e-01,   1.26391280e+00,  -9.48799563e-01,
#         -7.79099318e-01,   2.00268261e+00,  -8.90495881e-01,
#         -8.59210993e-03,  -1.54948191e+00,   1.39577855e+00,
#          2.43904610e+02,   3.47694754e+00,  -3.14999583e-01,
#          6.33069119e-02])

# start_params.append([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
#          3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
#         -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
#         -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
#          7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
#         -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
#          1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
#          2.18327852e+00,   6.63152141e-02, 0.0])

start_params.append([ -2.95016477e+00,   3.78177969e-04,  -5.21095611e+02,
         3.09845244e+02,  -4.69222440e+02,   3.95186037e+02,
        -1.11630659e+02,   5.36915127e+00,   1.52984702e+01,
        -1.54915170e+01,   7.53107481e+00,  -2.98591742e+00,
         7.55346307e-01,  -1.00678488e+00,   1.96634582e+00,
        -8.71560927e-01,  -1.41820832e-01,  -1.46508000e+00,
         1.42936365e+00,   7.23805795e+01,   0.0, 9.18071460e-01,
         2.04259992e+00,   6.28575567e-02,   7.62804667e-03])

multi_pars = start_params

In [53]:
def optimize_EAM_mp(pars, targets, stats, utot_func):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [ ]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1])]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [-2.95016477, 0.000378177969, -521.095611, 309.845244, -469.22244, 395.186037, -111.630659, 5.36915127, 15.2984702, -15.491517, 7.53107481, -2.98591742, 0.755346307, -1.00678488, 1.96634582, -0.871560927, -0.141820832, -1.46508, 1.42936365, 72.3805795, 0.0, 0.91807146, 2.04259992, 0.0628575567, 0.00762804667])
loss 2.8642886588e-06 2.8642886588e-06 0.0
('Best params:', [(2.8642886588038736e-06, [-2.95016477, 0.000378177969, -521.095611, 309.845244, -469.22244, 395.186037, -111.630659, 5.36915127, 15.2984702, -15.491517, 7.53107481, -2.98591742, 0.755346307, -1.00678488, 1.96634582, -0.871560927, -0.141820832, -1.46508, 1.42936365, 72.3805795, 0.0, 0.91807146, 2.04259992, 0.0628575567, 0.00762804667])])
('ieam:', 0)
loss 2.8642886588e-06 2.8642886588e-06 0.0
loss 0.00173124258004 0.00173124258004 0.0
loss 3.10499293372e-06 3.10499293372e-06 0.0
loss 2.86448518722e-06 2.86448518722e-06 0.0
loss 3.46206776036e-06 3.46206776036e-06 0.0
loss 2.96723315666e-05 2.967233156

loss 1.13829618704e-05 1.13829618704e-05 0.0
loss 5.02946387398e-06 5.02946387398e-06 0.0
loss 1.40494428722e-05 1.40494428722e-05 0.0
loss 4.66736080166e-06 4.66736080166e-06 0.0
loss 1.41921248989e-05 1.41921248989e-05 0.0
loss 4.46117963833e-06 4.46117963833e-06 0.0
loss 1.41823865076e-05 1.41823865076e-05 0.0
loss 4.26159303447e-06 4.26159303447e-06 0.0
loss 7.82688346768e-06 7.82688346768e-06 0.0
loss 9.92785126156e-06 9.92785126156e-06 0.0
loss 4.24423051232e-06 4.24423051232e-06 0.0
loss 8.36831773884e-06 8.36831773884e-06 0.0
loss 4.27421898119e-06 4.27421898119e-06 0.0
loss 8.64304747841e-06 8.64304747841e-06 0.0
loss 4.23510264907e-06 4.23510264907e-06 0.0
loss 7.86650730535e-06 7.86650730535e-06 0.0
loss 4.11719029984e-06 4.11719029984e-06 0.0
loss 9.41084100791e-06 9.41084100791e-06 0.0
loss 3.96325938917e-06 3.96325938917e-06 0.0
loss 1.01317216586e-05 1.01317216586e-05 0.0
loss 3.80160601299e-06 3.80160601299e-06 0.0
loss 6.06729429593e-06 6.06729429593e-06 0.0
loss 6.999

loss 2.86917315655e-06 2.86917315655e-06 0.0
loss 2.88657315051e-06 2.88657315051e-06 0.0
loss 2.86893068772e-06 2.86893068772e-06 0.0
loss 2.88436889229e-06 2.88436889229e-06 0.0
loss 2.86901076133e-06 2.86901076133e-06 0.0
loss 2.88745655975e-06 2.88745655975e-06 0.0
loss 2.86844292149e-06 2.86844292149e-06 0.0
loss 2.88769863269e-06 2.88769863269e-06 0.0
loss 2.86780677962e-06 2.86780677962e-06 0.0
loss 2.87804914874e-06 2.87804914874e-06 0.0
loss 2.87721660055e-06 2.87721660055e-06 0.0
loss 2.88012614529e-06 2.88012614529e-06 0.0
loss 2.86834633474e-06 2.86834633474e-06 0.0
loss 2.88101257305e-06 2.88101257305e-06 0.0
loss 2.8681823087e-06 2.8681823087e-06 0.0
loss 2.88310067289e-06 2.88310067289e-06 0.0
loss 2.86783735481e-06 2.86783735481e-06 0.0
loss 2.87859433385e-06 2.87859433385e-06 0.0
loss 2.86811240509e-06 2.86811240509e-06 0.0
loss 2.88434939902e-06 2.88434939902e-06 0.0
loss 2.86743803116e-06 2.86743803116e-06 0.0
loss 2.87626273091e-06 2.87626273091e-06 0.0
loss 2.88358

loss 2.86448570315e-06 2.86448570315e-06 0.0
loss 2.86435227832e-06 2.86435227832e-06 0.0
loss 2.86446256817e-06 2.86446256817e-06 0.0
loss 2.86447790755e-06 2.86447790755e-06 0.0
loss 2.86434891616e-06 2.86434891616e-06 0.0
loss 2.86443313302e-06 2.86443313302e-06 0.0
loss 2.86446847095e-06 2.86446847095e-06 0.0
loss 2.86434390865e-06 2.86434390865e-06 0.0
loss 2.86445121764e-06 2.86445121764e-06 0.0
loss 2.8645196404e-06 2.8645196404e-06 0.0
loss 2.86433587228e-06 2.86433587228e-06 0.0
loss 2.86444557592e-06 2.86444557592e-06 0.0
loss 2.8644169611e-06 2.8644169611e-06 0.0
loss 2.86443708142e-06 2.86443708142e-06 0.0
loss 2.86444024084e-06 2.86444024084e-06 0.0
loss 2.86433693652e-06 2.86433693652e-06 0.0
loss 2.86447872755e-06 2.86447872755e-06 0.0
loss 2.86433203157e-06 2.86433203157e-06 0.0
loss 2.86445571677e-06 2.86445571677e-06 0.0
loss 2.86433466984e-06 2.86433466984e-06 0.0
loss 2.86453492899e-06 2.86453492899e-06 0.0
loss 2.86432577317e-06 2.86432577317e-06 0.0
loss 2.8644102

loss 2.86429086766e-06 2.86429086766e-06 0.0
loss 2.86429302258e-06 2.86429302258e-06 0.0
loss 2.86428911437e-06 2.86428911437e-06 0.0
loss 2.8642914885e-06 2.8642914885e-06 0.0
loss 2.86428918284e-06 2.86428918284e-06 0.0
loss 2.86429221945e-06 2.86429221945e-06 0.0
loss 2.86428908982e-06 2.86428908982e-06 0.0
loss 2.86429236756e-06 2.86429236756e-06 0.0
loss 2.86428906827e-06 2.86428906827e-06 0.0
loss 2.86429112196e-06 2.86429112196e-06 0.0
loss 2.86429159539e-06 2.86429159539e-06 0.0
loss 2.86428910709e-06 2.86428910709e-06 0.0
loss 2.86429107662e-06 2.86429107662e-06 0.0
loss 2.86428995042e-06 2.86428995042e-06 0.0
loss 2.86429069231e-06 2.86429069231e-06 0.0
loss 2.86429117594e-06 2.86429117594e-06 0.0
loss 2.8642889863e-06 2.8642889863e-06 0.0
loss 2.86429123672e-06 2.86429123672e-06 0.0
loss 2.86428891616e-06 2.86428891616e-06 0.0
loss 2.86429217488e-06 2.86429217488e-06 0.0
loss 2.86428878781e-06 2.86428878781e-06 0.0
loss 2.86429125025e-06 2.86429125025e-06 0.0
loss 2.8642888

...

loss 2.83729247387e-06 2.83729247387e-06 0.0
loss 2.83735009192e-06 2.83735009192e-06 0.0
loss 2.83730757977e-06 2.83730757977e-06 0.0
loss 2.83735492372e-06 2.83735492372e-06 0.0
loss 2.83732017492e-06 2.83732017492e-06 0.0
loss 2.83734508096e-06 2.83734508096e-06 0.0
loss 2.83735075331e-06 2.83735075331e-06 0.0
loss 2.83731812752e-06 2.83731812752e-06 0.0
loss 2.83733336671e-06 2.83733336671e-06 0.0
loss 2.83732398566e-06 2.83732398566e-06 0.0
loss 2.83735923144e-06 2.83735923144e-06 0.0
loss 2.83731640722e-06 2.83731640722e-06 0.0
loss 2.83729368323e-06 2.83729368323e-06 0.0
loss 2.83731369831e-06 2.83731369831e-06 0.0
loss 2.8373008161e-06 2.8373008161e-06 0.0
loss 2.83731559852e-06 2.83731559852e-06 0.0
loss 2.83732607452e-06 2.83732607452e-06 0.0
loss 2.83731109784e-06 2.83731109784e-06 0.0
loss 2.83730568511e-06 2.83730568511e-06 0.0
loss 2.83729432081e-06 2.83729432081e-06 0.0
loss 2.83729036519e-06 2.83729036519e-06 0.0
loss 2.83728068477e-06 2.83728068477e-06 0.0
loss 2.83730

loss 2.83701380834e-06 2.83701380834e-06 0.0
loss 2.83704372464e-06 2.83704372464e-06 0.0
loss 2.83702800173e-06 2.83702800173e-06 0.0
loss 2.83699131088e-06 2.83699131088e-06 0.0
loss 2.83699756488e-06 2.83699756488e-06 0.0
loss 2.83701309113e-06 2.83701309113e-06 0.0
loss 2.83701192092e-06 2.83701192092e-06 0.0
loss 2.83698684254e-06 2.83698684254e-06 0.0
loss 2.8370154907e-06 2.8370154907e-06 0.0
loss 2.83699495452e-06 2.83699495452e-06 0.0
loss 2.83701675298e-06 2.83701675298e-06 0.0
loss 2.83701261438e-06 2.83701261438e-06 0.0
loss 2.83697061946e-06 2.83697061946e-06 0.0
loss 2.83697153927e-06 2.83697153927e-06 0.0
loss 2.8370161206e-06 2.8370161206e-06 0.0
loss 2.83695685498e-06 2.83695685498e-06 0.0
loss 2.83694338659e-06 2.83694338659e-06 0.0
loss 2.83695634348e-06 2.83695634348e-06 0.0
loss 2.83697909947e-06 2.83697909947e-06 0.0
loss 2.83695641302e-06 2.83695641302e-06 0.0
loss 2.83695911092e-06 2.83695911092e-06 0.0
loss 2.83696603495e-06 2.83696603495e-06 0.0
loss 2.8369515

loss 2.83633418959e-06 2.83633418959e-06 0.0
loss 2.8361456703e-06 2.8361456703e-06 0.0
loss 2.8361826345e-06 2.8361826345e-06 0.0
loss 2.8360167674e-06 2.8360167674e-06 0.0
loss 2.83601390547e-06 2.83601390547e-06 0.0
loss 2.83623856096e-06 2.83623856096e-06 0.0
loss 2.83603621711e-06 2.83603621711e-06 0.0
loss 2.8360750482e-06 2.8360750482e-06 0.0
loss 2.83613694393e-06 2.83613694393e-06 0.0
loss 2.8360084524e-06 2.8360084524e-06 0.0
loss 2.83600659947e-06 2.83600659947e-06 0.0
loss 2.8362138806e-06 2.8362138806e-06 0.0
loss 2.83601764319e-06 2.83601764319e-06 0.0
loss 2.83606306275e-06 2.83606306275e-06 0.0
loss 2.83612301906e-06 2.83612301906e-06 0.0
loss 2.8359744122e-06 2.8359744122e-06 0.0
loss 2.83616664737e-06 2.83616664737e-06 0.0
loss 2.83617633976e-06 2.83617633976e-06 0.0
loss 2.83600369542e-06 2.83600369542e-06 0.0
loss 2.83588396576e-06 2.83588396576e-06 0.0
loss 2.83594126033e-06 2.83594126033e-06 0.0
loss 2.83601776639e-06 2.83601776639e-06 0.0
loss 2.83612918002e-06 2

loss 2.83525457766e-06 2.83525457766e-06 0.0
loss 2.83525553797e-06 2.83525553797e-06 0.0
loss 2.83528311097e-06 2.83528311097e-06 0.0
loss 2.83537345053e-06 2.83537345053e-06 0.0
loss 2.83524191358e-06 2.83524191358e-06 0.0
loss 2.83543167514e-06 2.83543167514e-06 0.0
loss 2.83523658262e-06 2.83523658262e-06 0.0
loss 2.8351251172e-06 2.8351251172e-06 0.0
loss 2.8350953732e-06 2.8350953732e-06 0.0
loss 2.83516909832e-06 2.83516909832e-06 0.0
loss 2.83528574116e-06 2.83528574116e-06 0.0
loss 2.83513984884e-06 2.83513984884e-06 0.0
loss 2.83510237795e-06 2.83510237795e-06 0.0
loss 2.83512117577e-06 2.83512117577e-06 0.0
loss 2.83536584992e-06 2.83536584992e-06 0.0
loss 2.83519741394e-06 2.83519741394e-06 0.0
loss 2.8352341364e-06 2.8352341364e-06 0.0
loss 2.83516613896e-06 2.83516613896e-06 0.0
loss 2.83528437787e-06 2.83528437787e-06 0.0
loss 2.83518399024e-06 2.83518399024e-06 0.0
loss 2.83521613836e-06 2.83521613836e-06 0.0
loss 2.83528663303e-06 2.83528663303e-06 0.0
loss 2.835191674

loss 2.83486992145e-06 2.83486992145e-06 0.0
loss 2.83488668951e-06 2.83488668951e-06 0.0
loss 2.83491598804e-06 2.83491598804e-06 0.0
loss 2.8348772906e-06 2.8348772906e-06 0.0
loss 2.83487191019e-06 2.83487191019e-06 0.0
loss 2.83488754727e-06 2.83488754727e-06 0.0
loss 2.8348573458e-06 2.8348573458e-06 0.0
loss 2.83488733491e-06 2.83488733491e-06 0.0
loss 2.83486887813e-06 2.83486887813e-06 0.0
loss 2.8348992195e-06 2.8348992195e-06 0.0
loss 2.83488933846e-06 2.83488933846e-06 0.0
loss 2.83489684584e-06 2.83489684584e-06 0.0
loss 2.83490108562e-06 2.83490108562e-06 0.0
loss 2.83487112916e-06 2.83487112916e-06 0.0
loss 2.83486168963e-06 2.83486168963e-06 0.0
loss 2.83485631801e-06 2.83485631801e-06 0.0
loss 2.83489455775e-06 2.83489455775e-06 0.0
loss 2.8349107882e-06 2.8349107882e-06 0.0
loss 2.83486593672e-06 2.83486593672e-06 0.0
loss 2.83486553038e-06 2.83486553038e-06 0.0
loss 2.83489003477e-06 2.83489003477e-06 0.0
loss 2.83489127394e-06 2.83489127394e-06 0.0
loss 2.83486320472

In [60]:
params_output = [(8.9710060363567114e-07, np.array([ -4.18353755e+00,  -9.63668936e-03,  -1.00171035e+02,
         1.90025394e+01,   2.15938317e+01,  -1.39759833e+01,
         2.16332289e+01,  -3.29542126e+00,   1.70455674e+00,
         1.41347064e+00,  -9.02958785e-01,  -8.62309098e-01,
         1.95964588e+00,  -8.70527088e-01,   3.22342700e-02,
        -1.53866121e+00,   1.37095441e+00,  -4.32896107e+01,
         4.64461212e+00,   3.23329113e-01,   5.82061842e-02]))]

In [73]:
params_output = [(8.829397265807305e-07, np.array([ -4.33954858e+00,  -1.02215197e-02,  -1.12590032e+02,
         3.06702045e+01,   1.74951979e+01,  -1.33298120e+01,
         2.13899957e+01,  -3.70881631e+00,   1.56452640e+00,
         1.27789071e+00,  -8.54623702e-01,  -8.39818015e-01,
         1.95990543e+00,  -8.75948596e-01,   1.81927625e-02,
        -1.54683863e+00,   1.38614460e+00,  -4.28747460e+01,
         4.18974818e+00,   1.56622590e-01,   5.72961538e-02,
         3.46319317e+00]))]

In [114]:
params_output = [(8.7421414571196454e-07, np.array([ -4.31249172e+00,  -9.93971047e-03,  -3.76018390e+01,
        -1.14759006e+02,   1.72557600e+01,   2.66437414e+01,
        -1.50817090e+01,   2.22342349e+01,  -3.62963119e+00,
         1.62230813e+00,   1.18480877e+00,  -8.52354746e-01,
        -8.38460594e-01,   1.98957701e+00,  -8.86622667e-01,
         1.66082225e-03,  -1.54824194e+00,   1.39761513e+00,
        -5.06767503e+01,   4.47029066e+00,   9.10205250e-02,
         5.87302830e-02,   1.82639066e+00]))]

In [39]:
params_output = [(8.8257346852300666e-07, np.array([ -4.36548916e+00,  -1.11867827e-02,  -1.94093020e+01,
        -1.40195282e+02,   4.53864231e+01,   1.49756180e+01,
        -1.36031385e+01,   2.21581200e+01,  -4.61501041e+00,
         2.06360250e+00,   1.12795645e+00,  -8.01200744e-01,
        -8.58426028e-01,   1.95860157e+00,  -8.68817546e-01,
         2.43936831e-02,  -1.54555297e+00,   1.38005891e+00,
        -5.22754243e+01,   3.98209114e+00,   3.07369702e-01,
         5.51658454e-02,   1.58601447e+00]))]

In [40]:
params_output = [(2.8925274000958436e-06, np.array([ -3.09187534e+00,   1.19622206e-04,  -5.67875917e+02,
         3.21802084e+02,  -4.71317094e+02,   3.98296604e+02,
        -1.13270858e+02,   6.03114401e+00,   1.48067908e+01,
        -1.57531660e+01,   7.80863462e+00,  -2.99246651e+00,
         7.33596451e-01,  -9.50794219e-01,   1.94186078e+00,
        -8.83596676e-01,  -1.33608784e-01,  -1.46694544e+00,
         1.42447434e+00,   7.53865380e+01,   3.29733034e-01,
         2.18327852e+00,   6.63152141e-02]))]

In [62]:
# convert lists of optimal parameters to dictionaries
params_optimal = []
for pars in params_output:
    #params_optimal.append(to_param_dict(pars, [2, 15, 4, 0]))
    params_optimal.append(to_param_dict(pars[1], stats['hyperparams']))

In [64]:
# store hyperparameters and output parameters in a single file
try:
    with open(os.path.join(working, 'params_store'+'.pickle'), 'rb') as fi:
        params_store = pickle.load(fi)    
except IOError:
    print('no file')
    params_store = []
    
params_store.extend(params_optimal)

with open(os.path.join(working, "params_store.pickle"), 'wb') as fo:
    pickle.dump(params_store, fo, protocol=2)

no file


In [65]:
with open(os.path.join(working, 'params_store' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [66]:
retreived_params

[{'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': array([-43.2896107 ,   4.64461212,   0.32332911,   0.05820618]),
   'embed': array([-4.18353755, -0.00963669]),
   'lrcorr': array([], dtype=float64),
   'pair': array([ -1.00171035e+02,   1.90025394e+01,   2.15938317e+01,
           -1.39759833e+01,   2.16332289e+01,  -3.29542126e+00,
            1.70455674e+00,   1.41347064e+00,  -9.02958785e-01,
           -8.62309098e-01,   1.95964588e+00,  -8.70527088e-01,
            3.22342700e-02,  -1.53866121e+00,   1.37095441e+00])}}]